# Библиотеки

In [1]:
import pandas as pd
import praw
import snscrape.modules.twitter as sntwitter
import datetime
from tqdm.notebook import tqdm
import cryptocompare

import getpass

api_key = getpass.getpass('Api Key for CryptoCompare parsing:')
api_key = '&api_key=a060dc55e20b43f88610b534de61261c77e13940f3e50191986a84f67da06218'


client_id = getpass.getpass('Cliend id for Reddit parsing:')
client_id='gYuNiBS7_keBx3UmPQh-Nw'


client_secret = getpass.getpass('Cliend secret for Reddit parsing:')
client_secret='I3oLaghUnEk9IgmSRAhsGrp3QKanNQ'



from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 100% !important; } </style>'))

# Парсинг

## Reddit

In [2]:
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent='ParsBars')

posts_total = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']

for x in subreddit_list:
    posts = []
    ml_subreddit = reddit.subreddit(x).hot(limit=1000000)
    sub_c = reddit.subreddit(x)
    sub_c = sub_c.subscribers
    
    for post in tqdm(ml_subreddit):
        posts_loc = []
        posts_loc.append([post.author,post.title, post.score, post.id, post.subreddit, post.num_comments, post.selftext, datetime.datetime.fromtimestamp(post.created)])
        submission = reddit.submission(id=post.id)
        comm = []
        scor = []
        aut = []
        tim = []
        submission.comments.replace_more(limit=0)
        for top_level_comment in submission.comments:
            comm += [top_level_comment.body]
            scor += [top_level_comment.score]
            aut += [top_level_comment.author]
            tim += [datetime.datetime.fromtimestamp(top_level_comment.created)]
        df = pd.DataFrame([comm,scor,aut,tim],index=['comment','comment_score','aut','tim']).T.sort_values(by='comment_score',ascending=False).iloc[:10]
        shap = min(df.shape[0],10)
        comm_df = pd.DataFrame(df.iloc[:,0].tolist()+df.iloc[:,1].tolist()+df.iloc[:,2].tolist() + df.iloc[:,3].tolist() \
                               + [sum(scor),sub_c],index= ['top {} comment'.format(x) for x in range(1,shap+1)]\
                               + ['top {} comment score'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment author'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment time'.format(x) for x in range(1,shap+1)] \
                               + ['total comments score','subreddit subscribers']).T
        posts +=[pd.concat([pd.DataFrame(posts_loc,columns=['author','title', 'score', 'id', 'subreddit', 'num_comments', 'body', 'created']),comm_df],axis=1)]
    
    posts = pd.concat(posts)
    posts_total += [posts]
posts_total = pd.concat(posts_total).to_csv('reddit.csv')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Twitter

In [ ]:
maxTweets = 1000000

posts_total = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']
for x in subreddit_list:
    tweets_list = []
    try:
        for i,tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper(x).get_items())):
            if i>maxTweets:
                break
            posts_total.append({'date':tweet.date, 
                                 'id':tweet.id, 
                                 'content':tweet.content, 
                                 'username':tweet.username})
    except:
        pass
    
    tweets_df = pd.DataFrame(tweets_list)
    tweets_df['subreddit'] = x
    posts_total += [tweets_df]
posts_total = pd.concat(posts_total).to_csv('twitter.csv')

0it [00:00, ?it/s]

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=Bitcoin&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaIwKO1wszt-icWiIC9oYT_2v8nEnEVi4TCARWAiXoYBFVTRVI1ARWE-wEVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: non-200 status code, retrying
Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_

0it [00:00, ?it/s]

## Bitcoin price 

 НАЙТИ ВРЕМЯ ДЛЯ ПАРСИНГА ЦЕН

### Daily

In [ ]:
assets = 'BTC'

# В целом по биржам
url_daily_exchange_vol = 'https://min-api.cryptocompare.com/data/exchange/histoday?tsym=USD&limit=60' + api_key
data_exchange = pd.DataFrame(requests.get(url_daily_exchange_vol).json()['Data'])


url_daily_pair = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit=60'.format(assets) + api_key
# По активу
url_daily_symbol_vol = 'https://min-api.cryptocompare.com/data/symbol/histoday?fsym={}&tsym=USD&limit=60'.format(assets) + api_key

try:
    data_main = pd.DataFrame(requests.get(url_daily_pair).json()['Data']['Data'])
    data_vol = pd.DataFrame(requests.get(url_daily_symbol_vol).json()['Data'])
    data = pd.concat([data_main,data_vol,data_exchange],axis=1)
    
    daily_mass.update({assets:data})
except KeyError:
    print("I can't find "+assets + ' in USD')

### Hourly

In [ ]:
# В целом по биржам
url_hourly_exchange_vol = 'https://min-api.cryptocompare.com/data/exchange/histohour?tsym=USD&limit=168' + api_key
data_exchange = pd.DataFrame(requests.get(url_hourly_exchange_vol).json()['Data'])


url_hourly_pair = 'https://min-api.cryptocompare.com/data/v2/histohour?fsym={}&tsym=USD&limit=168'.format(assets) + api_key
# По активу
url_hourly_symbol_vol = 'https://min-api.cryptocompare.com/data/symbol/histohour?fsym={}&tsym=USD&limit=168'.format(assets) + api_key

try:
    data_main = pd.DataFrame(requests.get(url_hourly_pair).json()['Data']['Data'])
    data_vol = pd.DataFrame(requests.get(url_hourly_symbol_vol).json()['Data'])
    data = pd.concat([data_main,data_vol,data_exchange],axis=1)
    
    hourly_mass.update({assets:data})
except KeyError:
    print("I can't find "+assets + ' in USD')

### Minutely

In [ ]:
url_minutely_pair = 'https://min-api.cryptocompare.com/data/v2/histominute?fsym={}&tsym=USD&limit=1440'.format(assets) + api_key

try:
    data_main = pd.DataFrame(requests.get(url_minutely_pair).json()['Data']['Data'])
    data = data_main
    
    minutely_mass.update({assets:data})
except KeyError:
    print("I can't find "+assets + ' in USD')